# Web **Scraping**


Kode ini bertujuan untuk melakukan web scraping pada website https://www.indonesiapostcode.com/location/ untuk mengambil daftar nama provinsi, kota/kabupaten di dalamnya, serta tautan (link) ke halaman detail untuk setiap kota/kabupaten. Hasilnya kemudian disimpan ke dalam file CSV bernama DAFTAR_PROVINSI_KABUPATEN_DAN_LINK.csv.

Penjelasan langkah demi langkah:

1. Import Library: Mengimpor library yang dibutuhkan (requests untuk mengambil data dari web, BeautifulSoup untuk mem-parsing HTML, pandas untuk mengelola data dalam DataFrame, dan time untuk memberi jeda).
2. Konfigurasi URL dan Selector: Mendefinisikan URL utama situs dan selector CSS (h4.media-heading a) yang digunakan untuk menemukan tautan provinsi dan kota/kabupaten.

3. List Penampung: Membuat list kosong hasil_data untuk menyimpan data yang berhasil di-scrape.
* Mengambil Daftar Provinsi (Level 1):
* Mengakses URL utama.
* Menggunakan BeautifulSoup untuk mem-parsing halaman.
* Mencari semua tautan provinsi menggunakan selector CSS yang ditentukan.
* Jika tautan provinsi ditemukan, loop akan dimulai untuk setiap provinsi.
4. Memproses Provinsi dan Mengambil Kota/Kabupaten (Level 2):
Untuk setiap tautan provinsi, kode mengambil nama provinsi dan URL lengkapnya.
* Mengakses URL provinsi tersebut.
* Mem-parsing halaman provinsi.
* Mencari semua tautan kota/kabupaten di halaman provinsi menggunakan selector CSS yang sama.
* Jika tautan kota/kabupaten ditemukan, loop akan berjalan untuk setiap kota/kabupaten.
* Mengambil nama kota/kabupaten dan href tautannya.
* Membangun tautan lengkap ke halaman detail kota/kabupaten.
* Menambahkan data (Provinsi, Kota/Kabupaten, Link) ke list hasil_data.
* Memberi jeda 1 detik setelah memproses setiap provinsi untuk menghindari pemblokiran.
5. Menyimpan Hasil ke CSV:
* Setelah selesai memproses semua provinsi dan kota/kabupaten, kode membuat DataFrame Pandas dari list hasil_data.
* Menyimpan DataFrame ke file CSV bernama DAFTAR_PROVINSI_KABUPATEN.csv.
* Menampilkan 5 baris pertama dari DataFrame sebagai konfirmasi.

In [ ]:
# Pastikan Anda sudah meng-install library ini:
# pip install requests beautifulsoup4 pandas lxml

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

main_url = "https://www.indonesiapostcode.com/location/"
base_url = "https://www.indonesiapostcode.com"

# Selector CSS dari website indonesiapostcode
CSS_SELECTOR_LINK = "h4.media-heading a"

# List untuk menampung hasil akhir (Provinsi, Kabupaten, Link)
hasil_data = []

try:
    print("LEVEL 1: Mengambil daftar provinsi...")
    response_main = requests.get(main_url)
    soup_main = BeautifulSoup(response_main.text, 'html.parser')

    # 1. Gunakan .select() untuk menemukan semua link provinsi
    provinsi_links = soup_main.select(CSS_SELECTOR_LINK)

    if not provinsi_links:
        print("Tidak bisa menemukan link provinsi. Selector mungkin berubah.")
    else:
        for link_prov in provinsi_links:
            nama_provinsi = link_prov.text.strip()
            url_provinsi = base_url + link_prov['href']

            if 'location' not in url_provinsi: continue

            print(f"\nLEVEL 2: Memproses provinsi: {nama_provinsi}...")

            response_prov = requests.get(url_provinsi)
            soup_prov = BeautifulSoup(response_prov.text, 'html.parser')

            # 2. Gunakan selector YANG SAMA untuk menemukan link kota/kabupaten
            kabupaten_links = soup_prov.select(CSS_SELECTOR_LINK)

            if not kabupaten_links:
                print(f"  -> Tidak ada kabupaten ditemukan di {nama_provinsi}.")
            else:
                for link_kab in kabupaten_links:
                    nama_kabupaten = link_kab.text.strip()
                    href_kabupaten = link_kab['href'] # Ambil href untuk membangun link

                    if 'location' not in href_kabupaten: continue

                    # ======================================================
                    # PERUBAHAN: LANGSUNG SIMPAN, TIDAK MASUK KE LEVEL 3
                    # ======================================================
                    # Bangun link lengkap berdasarkan href kabupaten
                    link_lengkap = base_url + href_kabupaten

                    print(f"  -> Ditemukan: {nama_kabupaten}")
                    hasil_data.append({
                        'Provinsi': nama_provinsi,
                        'Kota_Kabupaten': nama_kabupaten,
                        'Link': link_lengkap # Tambahkan link ke data
                    })

            # Beri jeda 1 detik antar PROVINSI
            time.sleep(1)

    # 5. Simpan hasil akhir ke CSV
    if hasil_data:
        print("\n\nMenggabungkan semua data...")

        # Buat DataFrame langsung dari list 'hasil_data'
        df_final = pd.DataFrame(hasil_data)

        nama_file = "DAFTAR_PROVINSI_KABUPATEN.csv" # Ubah nama file CSV
        df_final.to_csv(nama_file, index=False)

        print(f"Selesai! Data disimpan di '{nama_file}'")
        print(df_final.head()) # Tampilkan 5 data pertama
    else:
        print("Tidak ada data yang berhasil diambil.")

except requests.RequestException as e:
    print(f"Terjadi error saat mengakses URL: {e}")
except Exception as e:
    print(f"Terjadi error: {e}")

In [ ]:
display(df_final.head(100))

In [ ]:
# Hitung jumlah provinsi unik
jumlah_provinsi = df_final['Provinsi'].nunique()
print(f"Jumlah provinsi di Indonesia adalah: {jumlah_provinsi}")

# Hitung jumlah kota/kabupaten (jumlah baris dalam DataFrame)
jumlah_kota_kabupaten = len(df_final)
print(f"Jumlah kota/kabupaten di Indonesia adalah: {jumlah_kota_kabupaten}")

In [ ]:
# Hitung jumlah kota/kabupaten per provinsi
jumlah_per_provinsi = df_final.groupby('Provinsi').size().reset_index(name='Jumlah_Kota_Kabupaten')

display(jumlah_per_provinsi)